In [1]:
from datasets import concatenate_datasets, load_from_disk
from transformers import BasicTokenizer, Trainer, EarlyStoppingCallback
import kagglehub
import torch
import base
import os

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [2]:
#my_glove = kagglehub.dataset_download("takuok/glove840b300dtxt")
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [ ]:
#GLOVE_FILE = f"{my_glove}/glove.840B.300d.txt"
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "dbpedia"

In [4]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits", f"~/data/{DATASET}/test-logits", f"~/data/{DATASET}/train-logits-augmented"]])
tokenizer = BasicTokenizer(do_lower_case=True)

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


In [6]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))
all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

: 

: 

In [ ]:
vocab = base.get_vocab(all_data_tokens)

In [ ]:
word_index = dict(zip(vocab, range(len(vocab))))

In [ ]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


In [ ]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

691158


In [ ]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 212978 words (478180) misses


In [ ]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

In [ ]:
train_padded_data = list(map(lambda x: base.padd(x,300), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,300), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,300), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,300), all_train_data_index))

In [ ]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

In [ ]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [ ]:
print(model)

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)


In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base", logging_dir=f"~/logs/{DATASET}/bilstm-base", lr=.0001,  epochs=3, batch_size=128)

In [ ]:
base.reset_seed()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.230100,0.069248,0.979964,0.980077,0.979964,0.979971
2,0.062700,0.056962,0.984196,0.984239,0.984196,0.984195
3,0.050200,0.051264,0.985866,0.985865,0.985866,0.985865


TrainOutput(global_step=10500, training_loss=0.11433484613327753, metrics={'train_runtime': 572.0667, 'train_samples_per_second': 2349.376, 'train_steps_per_second': 18.355, 'total_flos': 0.0, 'train_loss': 0.11433484613327753, 'epoch': 3.0})

In [ ]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.05017419531941414,
 'eval_accuracy': 0.9858714285714286,
 'eval_precision': 0.9858700628880533,
 'eval_recall': 0.9858714285714286,
 'eval_f1': 0.9858694017333223,
 'eval_runtime': 15.9018,
 'eval_samples_per_second': 4402.007,
 'eval_steps_per_second': 34.399,
 'epoch': 3.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base.pth")

In [ ]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill", lr=.0001,  epochs=3, batch_size=128, lambda_param=.75, temp=5)

In [ ]:
base.reset_seed()

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.055600,0.190455,0.979920,0.979938,0.979920,0.979907
2,0.215600,0.142569,0.982589,0.982629,0.982589,0.982561
3,0.175600,0.122276,0.984527,0.984513,0.984527,0.984513


TrainOutput(global_step=10500, training_loss=0.48225953311011904, metrics={'train_runtime': 655.5445, 'train_samples_per_second': 2050.204, 'train_steps_per_second': 16.017, 'total_flos': 0.0, 'train_loss': 0.48225953311011904, 'epoch': 3.0})

In [ ]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.12285516411066055,
 'eval_accuracy': 0.9841857142857143,
 'eval_precision': 0.984173896296803,
 'eval_recall': 0.9841857142857143,
 'eval_f1': 0.9841690832527004,
 'eval_runtime': 14.1801,
 'eval_samples_per_second': 4936.49,
 'eval_steps_per_second': 38.575,
 'epoch': 3.0}

In [ ]:
torch.save(student_model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill.pth")

In [ ]:
model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-aug", logging_dir=f"~/logs/{DATASET}/bilstm-base-aug", lr=.0001,  epochs=3, batch_size=128)

In [ ]:
base.reset_seed()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.208000,0.060120,0.983232,0.983234,0.983232,0.983224
2,0.076100,0.050490,0.985821,0.985832,0.985821,0.985816
3,0.062100,0.047384,0.986777,0.986779,0.986777,0.986777


TrainOutput(global_step=20610, training_loss=0.11540887454133308, metrics={'train_runtime': 1423.2056, 'train_samples_per_second': 1853.458, 'train_steps_per_second': 14.481, 'total_flos': 0.0, 'train_loss': 0.11540887454133308, 'epoch': 3.0})

In [ ]:
model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.046175818890333176,
 'eval_accuracy': 0.9867428571428571,
 'eval_precision': 0.9867385036592475,
 'eval_recall': 0.9867428571428573,
 'eval_f1': 0.9867395869986717,
 'eval_runtime': 21.7972,
 'eval_samples_per_second': 3211.424,
 'eval_steps_per_second': 25.095,
 'epoch': 3.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-base-aug.pth")

In [ ]:
student_model = base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=14)

In [ ]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-aug", remove_unused_columns=False, logging_dir=f"~/logs/{DATASET}/bilstm-distill-aug", lr=.0001,  epochs=3, batch_size=128, lambda_param=.75, temp=5)

In [ ]:
base.reset_seed()

In [ ]:
trainer = base.DistilTrainer(
    student_model=student_model,
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.871700,0.168142,0.980518,0.980552,0.980518,0.980480
2,0.241700,0.123618,0.983929,0.983950,0.983929,0.983908
3,0.198400,0.107652,0.985446,0.985431,0.985446,0.985435


TrainOutput(global_step=20610, training_loss=0.4372475081309134, metrics={'train_runtime': 1513.0336, 'train_samples_per_second': 1743.419, 'train_steps_per_second': 13.622, 'total_flos': 0.0, 'train_loss': 0.4372475081309134, 'epoch': 3.0})

In [ ]:
student_model.eval()

BiLSTMClassifier(
  (embedding): Embedding(691160, 300)
  (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=600, out_features=400, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=400, out_features=14, bias=True)
)

In [ ]:
trainer.evaluate(test_data)

{'eval_loss': 0.10681172460317612,
 'eval_accuracy': 0.9854285714285714,
 'eval_precision': 0.9854093760634609,
 'eval_recall': 0.9854285714285714,
 'eval_f1': 0.9854123976057806,
 'eval_runtime': 23.2612,
 'eval_samples_per_second': 3009.306,
 'eval_steps_per_second': 23.516,
 'epoch': 3.0}

In [ ]:
torch.save(model.state_dict(), f"{os.path.expanduser('~')}/models/{DATASET}/bilstm-distill-aug.pth")